In [2]:
# INITS + IMPORTS
from typing import List
from openai import OpenAI
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import os
from dotenv import load_dotenv
from datasets import load_dataset
import pandas as pd
import dataclasses


BASE_URL = "http://199.94.61.113:8000/v1/"
API_KEY=api_key="yen.k@northeastern.edu:79wlaJxIMpzvkt61S8Xi"
assert BASE_URL is not None
assert API_KEY is not None

client = OpenAI(base_url=BASE_URL, api_key=API_KEY)

load_dotenv() 

# Set up the client id and client secret (you can find the client secret 
# from the spotify dev page)
CLIENT_ID = os.getenv("SPOTIFY_CLIENT_ID")
CLIENT_SECRET = os.getenv("SPOTIFY_CLIENT_SECRET")
REDIRECT_URI = 'http://localhost:8888/callback'
SCOPE = 'playlist-modify-public user-modify-playback-state user-top-read'

# Create a spotify object passing the client id, client secret, 
# redirct url (which doesn't matter, just set it as your local host
# as shown below), and scope
spotify = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=CLIENT_ID,
                                               client_secret=CLIENT_SECRET,
                                               redirect_uri=REDIRECT_URI,
                                               scope=SCOPE,
                                               cache_path=".cache-<your-username>"))


# https://huggingface.co/datasets/maharshipandya/spotify-tracks-dataset/viewer/default/train?q=4JyT0CxxEic1JhENHbXfR1
spotify_tracks_df = load_dataset("maharshipandya/spotify-tracks-dataset", split="train").to_pandas()

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:

# Function the the agent calls
def get_recommendations(self):
        limit = 10
        # Regardless of mood values you need seeds to get recommendations
        if len(self.seed_artists) == 0 and len(self.seed_tracks) == 0 and len(self.seed_genres) == 0:
            if self.target_valence == None and self.target_energy == None and self.target_danceability == None:
                print("No seeds or Mood provided. Returning top tracks.")
                top_tracks = spotify.current_user_top_tracks(limit=limit, offset=0, time_range='short_term')['items']
                formatted_top_tracks = []
                for track in top_tracks:
                    track_info = {'name': track['name'], 'artists': [artist['name'] for artist in track['artists']], 'uri': track['uri']}
                    formatted_top_tracks.append(track_info)
                return formatted_top_tracks


        recommendations = self.recommend_songs(
            seed_artists=self.seed_artists,
            seed_tracks=self.seed_tracks,
            seed_genres=self.seed_genres,
            valence=self.target_valence,
            energy=self.target_energy,
            danceability=self.target_danceability,
            limit=limit)
        return recommendations

# Function that gets songs from the spotify dataset
def recommend_songs(seed_artists=None, seed_tracks=None, seed_genres=None, valence=None, energy=None, danceability=None, limit=5):
    recommendations = spotify_tracks_df
    
    if seed_artists:
        artist_recommendations = recommendations[recommendations['artists'].isin(seed_artists)]
    else:
        artist_recommendations = pd.DataFrame()

    
    if seed_tracks:
        track_recommendations = recommendations[recommendations['track_id'].isin(seed_tracks)]
    else:
        track_recommendations = pd.DataFrame()

    if seed_genres:
        genre_recommendations = recommendations[recommendations['track_genre'].isin(seed_genres)]
    else:
        genre_recommendations = pd.DataFrame()
    

    # Use artist_recommendations if available, else track_recommendations, else the full dataset

    if not artist_recommendations.empty:
        recommendations = artist_recommendations
    elif not track_recommendations.empty:
        recommendations = track_recommendations
    elif not genre_recommendations.empty:
        recommendations = genre_recommendations
    else:
        recommendations = spotify_tracks_df

    if valence is not None:
        recommendations = recommendations[recommendations['valence'] >= valence]
    
    if energy is not None:
        recommendations = recommendations[recommendations['energy'] >= energy]
    
    if danceability is not None:
        recommendations = recommendations[recommendations['danceability'] >= danceability]

    if recommendations.empty:
        recommendations = spotify_tracks_df
    result = []
    for index, track in recommendations.head(limit).iterrows():
        artists = track['artists'].split(';')
        track_info = {'name': track['track_name'], 'artists': artists, 'uri': "spotify:track:" + track['track_id']}
        result.append(track_info)
    return result

seed_artists = ['Taylor Swift']
seed_tracks = ['Shake It Off']
seed_genres = ['pop']
target_valence = 0.5
target_energy = 0.5
target_danceability = 0.5

recommendations = recommend_songs(seed_artists=seed_artists, seed_tracks=seed_tracks, seed_genres=seed_genres, valence=target_valence, energy=target_energy, danceability=target_danceability)
print(recommendations)

[{'name': 'cardigan', 'artists': ['Taylor Swift'], 'uri': 'spotify:track:14MGlCeQzAJj3EeznSGeEC'}, {'name': 'Blank Space', 'artists': ['Taylor Swift'], 'uri': 'spotify:track:1u8c2t2Cy7UBoG4ArRcF5g'}]
